# Import

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import sqlite3

# Process HATCH Data
https://zenodo.org/records/10231865

In [2]:
raw_data = pd.read_csv('HATCH_dataset/HATCH_v1.5_Clean.csv')
year_columns = [col for col in raw_data.columns if col.isdigit()]
data = pd.melt(raw_data, id_vars=[col for col in raw_data.columns if col not in year_columns], 
                   value_vars=year_columns, var_name='Year', value_name='Value')
data['Year'] = data['Year'].astype(int)

# grouped_metrics = data.groupby('Region')['Country Name'].unique()
# for region, country in grouped_metrics.items():
#     print(f"{region}: {list(country)}")

len(data['Technology Name'].unique())

202

In [3]:
oecd_countries = [  # as per https://www.oecd.org/en/about/members-partners.html 
    'AUS',  # Australia
    'AUT',  # Austria
    'BEL',  # Belgium
    'CAN',  # Canada
    'CHL',  # Chile
    'COL',  # Colombia
    'CRI',  # Costa Rica
    'CZE',  # Czech Republic / Czechia
    'DNK',  # Denmark
    'EST',  # Estonia
    'FIN',  # Finland
    'FRA',  # France
    'DEU',  # Germany
    'GRC',  # Greece
    'HUN',  # Hungary
    'ISL',  # Iceland
    'IRL',  # Ireland
    'ISR',  # Israel
    'ITA',  # Italy
    'JPN',  # Japan
    'KOR',  # Korea
    'LVA',  # Latvia
    'LTU',  # Lithuania
    'LUX',  # Luxembourg
    'MEX',  # Mexico
    'NLD',  # Netherlands
    'NZL',  # New Zealand
    'NOR',  # Norway
    'POL',  # Poland
    'PRT',  # Portugal
    'SVK',  # Slovak Republic
    'SVN',  # Slovenia
    'ESP',  # Spain
    'SWE',  # Sweden
    'CHE',  # Switzerland
    'TUR',  # Turkey
    'GBR',  # United Kingdom
    'USA',  # United States
    'World'
]
g20_eu_countries = [
    # G20 Countries
    'ARG',  # Argentina
    'AUS',  # Australia
    'BRA',  # Brazil
    'CAN',  # Canada
    'CHN',  # China
    'FRA',  # France
    'DEU',  # Germany
    'IND',  # India
    'IDN',  # Indonesia
    'ITA',  # Italy
    'JPN',  # Japan
    'KOR',  # South Korea
    'MEX',  # Mexico
    'RUS',  # Russia
    'SAU',  # Saudi Arabia
    'ZAF',  # South Africa
    'TUR',  # Turkey
    'GBR',  # United Kingdom
    'USA',  # United States
    
    # EU Countries not already in G20 list
    'AUT',  # Austria
    'BEL',  # Belgium
    'BGR',  # Bulgaria
    'HRV',  # Croatia
    'CYP',  # Republic of Cyprus
    'CZE',  # Czech Republic
    'DNK',  # Denmark
    'EST',  # Estonia
    'FIN',  # Finland
    'GRC',  # Greece
    'HUN',  # Hungary
    'IRL',  # Ireland
    'LVA',  # Latvia
    'LTU',  # Lithuania
    'LUX',  # Luxembourg
    'MLT',  # Malta
    'NLD',  # Netherlands
    'POL',  # Poland
    'PRT',  # Portugal
    'ROU',  # Romania
    'SVK',  # Slovakia
    'SVN',  # Slovenia
    'ESP',  # Spain
    'SWE'   # Sweden
]
countries_dict = {
    # OECD Countries
    'AUS': 'Australia',
    'AUT': 'Austria',
    'BEL': 'Belgium',
    'CAN': 'Canada',
    'CHL': 'Chile',
    'COL': 'Colombia',
    'CRI': 'Costa Rica',
    'CZE': 'Czech Republic',
    'DNK': 'Denmark',
    'EST': 'Estonia',
    'FIN': 'Finland',
    'FRA': 'France',
    'DEU': 'Germany',
    'GRC': 'Greece',
    'HUN': 'Hungary',
    'ISL': 'Iceland',
    'IRL': 'Ireland',
    'ISR': 'Israel',
    'ITA': 'Italy',
    'JPN': 'Japan',
    'KOR': 'Korea',
    'LVA': 'Latvia',
    'LTU': 'Lithuania',
    'LUX': 'Luxembourg',
    'MEX': 'Mexico',
    'NLD': 'Netherlands',
    'NZL': 'New Zealand',
    'NOR': 'Norway',
    'POL': 'Poland',
    'PRT': 'Portugal',
    'SVK': 'Slovak Republic',
    'SVN': 'Slovenia',
    'ESP': 'Spain',
    'SWE': 'Sweden',
    'CHE': 'Switzerland',
    'TUR': 'Turkey',
    'GBR': 'United Kingdom',
    'USA': 'United States',

    # G20 Countries (not in OECD already)
    'ARG': 'Argentina',
    'BRA': 'Brazil',
    'CHN': 'China',
    'IND': 'India',
    'IDN': 'Indonesia',
    'RUS': 'Russia',
    'SAU': 'Saudi Arabia',
    'ZAF': 'South Africa',

    # EU Countries (not in OECD or G20 already)
    'BGR': 'Bulgaria',
    'HRV': 'Croatia',
    'CYP': 'Republic of Cyprus',
    'MLT': 'Malta',
    'ROU': 'Romania',
}

data_countries = data[data['Region'].isin(g20_eu_countries)].copy()    # choose whether to include OECD or G20+EU countries
data_countries['Region'] = data_countries['Region'].replace(countries_dict)

len(data_countries['Technology Name'].unique())

# [tech for tech in data_countries['Technology Name'].unique() if 'Offshore'.lower() in tech.lower()]

146

In [4]:
null_years = data_countries.groupby('Year').sum('Value')
null_years = null_years[null_years['Value'] == 0].index
data_countries = data_countries[~data_countries['Year'].isin(null_years)]
data_countries = data_countries.replace(0, np.nan)

def gap_filter(group):
    group = group.sort_values('Year')
    i0 = group['Value'].first_valid_index()
    il = group['Value'].last_valid_index()

    if i0 is not None:
        group = group.loc[i0:il]
        is_nan = group['Value'].isnull()
        nan_gaps = (is_nan != is_nan.shift()).cumsum()[is_nan].value_counts()

        return nan_gaps.max() if not nan_gaps.empty else 0
    return None
# print(data_countries.groupby('ID').apply(gap_filter).sort_values().iloc[-10:])

# data_filter = (
#     data_countries.groupby('ID').filter(lambda x: x['Value'].notnull().sum() >= 20)
# )
data_filter = data_countries.groupby('ID').filter(
    lambda group: (
        group['Value'].notnull().sum() >= 20 and    # series must have at least 20 entries
        (gap := gap_filter(group)) is not None and 
        gap <= 5                                    # series with gaps of more than 5 years are removed
    )
)
len(data_filter['Technology Name'].unique())

# print(data_filter.groupby('ID').apply(gap_filter).sort_values().iloc[-10:])
# data_filter[data_filter['ID'] == 'Passenger Cars_Total Number_PT'].plot(x='Year', y='Value')

98

In [5]:
data_filter['Metric'] = data_filter['Metric'].replace(to_replace=['Annual production', 'Cumulative total capacity', 'Net Total Capacity', 'Installed electricity capacity', 'Total Length'], 
                                                            value=['Annual Production', 'Cumulative Total Capacity', 'Cumulative Total Capacity', 'Installed Capacity', 'Cumulative Length'])

grouped_metrics = data_filter.groupby('Metric')['Technology Name'].unique()
print('##### metric: techs #####')
for metric, techs in grouped_metrics.items():
    print(f"{metric}: {list(techs)}")

grouped_metrics = data_filter.groupby('Metric')['Unit'].unique()
print('\n##### metric: units #####')
for metric, units in grouped_metrics.items():
    print(f"{metric}: {list(units)}")

keep_metrics = ['Total Number',
                'Annual Production',
                'Cumulative Total Capacity',
                'Cumulative Length',            # TODO - need to reconsider
                'Installed Capacity']

data_filter.loc[(data_filter['Metric'] == 'Cumulative Total Capacity') & (data_filter['Unit'] == 'kilometers'), 'Metric'] = 'Cumulative Length'
data_filter.loc[(data_filter['Metric'] == 'Cumulative Total Capacity') & (data_filter['Unit'] == '-'), 'Metric'] = 'Total Number'
data_metrics = data_filter[data_filter['Metric'].isin(keep_metrics)].copy()
len(data_metrics['Technology Name'].unique())

##### metric: techs #####
Annual Production: ['Beer Production', 'Gold', 'Cane Sugar', 'Zinc', 'Crude Oil', 'Silver', 'Sulphuric Acid', 'Nickel Production', 'Cement', 'Raw Steel Production', 'Primary Aluminum Production', 'Iron Ore', 'Primary Copper', 'Lead', 'Sand and Gravel|Industrial', 'Sand and Gravel|Construction', 'Salt Production', 'Cadmium Refining', 'Milk Production', 'Tin', 'Caustic Soda', 'Synthetic Filaments', 'Primary Bauxite Production', 'Nitric Acid', 'Hydrochloric Acid', 'Aquaculture Production', 'Capture Fisheries', 'Potash Fertilizer', 'Phosphate Fertilizer', 'Nitrogen Fertilizer', 'Renewable Power', 'Hydroelectricity', 'Oil Production', 'Nuclear Energy', 'Natural Gas Production', 'Coal Production', 'Electricity', 'Liquefied Natural Gas', 'All Biofuels', 'Copper|Refining', 'Copper|Mining', 'Air-Source Heat Pumps']
Cumulative Acreage: ['Insect-Resistant Cotton', 'Herbicide-Tolerant Soybeans', 'Herbicide-Tolerant Cotton', 'Herbicide-Tolerant Corn', 'Insect-Resistant Cor

72

In [6]:
# data_metrics[(data_metrics['Technology Name'].isin(['Crude Oil', 'Oil Production', 'Oil Refining Capacity'])) & 
#           (data_metrics['Country Name'] == 'Mexico') & (data_metrics['Year'].between(2000, 2010))]

# grouped_metrics = data_metrics.groupby('Technology Name')['Metric'].unique()
# for techs, metrics in grouped_metrics.items():
#     # if len(metrics) > 1:
#         print(f"{techs} - {metrics}")

remove_ids = ['Cellphones_Cumulative Total Capacity_World', 'Passenger Cars_Cumulative Total Capacity_World',   # repetitive or with data quality issues
              'Passenger Cars_Cumulative Total Capacity_US', 'Passenger Cars_Total Number_PT']

remove_techs = [    # nuclear weapons, space launches, oil production and nitrogen fertilizer techs were removed as per Edwards et al. 2023 (SI)
    'Aquaculture Production',
    'Beer Production',
    'Cane Sugar',
    'Capture Fisheries',
    'Caustic Soda',
    'Copper|Mining',
    'Copper|Refining'
    'Crude Oil',
    'Milk Production',
    'Nitrogen Fertilizer',
    'Nuclear Weapons',
    'Oil Production',
    'Postal Traffic',
    'Primary Bauxite Production',
    'Sand and Gravel|Industrial',
    'Space Launches'  
]

data_metrics = data_metrics[~data_metrics['ID'].isin(remove_ids)].copy()
data_clean = data_metrics[~data_metrics['Technology Name'].isin(remove_techs)].copy().dropna()
data_clean = data_clean.drop(['Country Name', 'Spatial Scale'], axis=1)
data_clean = data_clean.rename(columns={'ID': 'id', 'Region':'country', 'Technology Name': 'tech', 
                                        'Metric': 'metric', 'Unit': 'unit', 'Data Source': 'source',
                                        'Variable': 'variable', 'Year': 'year', 'Value': 'value'})

# data_clean.to_csv('hatch_clean.csv', index=False)

len(data_clean['tech'].unique())

58

In [7]:
filtered_techs = data_clean.groupby('tech')['value'].apply(lambda x: (x < 0.1).any())
techs_list = filtered_techs[filtered_techs].index
df = data_clean[data_clean['tech'].isin(techs_list)]

fig = px.box(df, x='value', y='tech', color='tech', log_x=True, points='all',
             height=1000, template='plotly_white', hover_data=['country', 'year', 'value', 'metric', 'unit'])
fig.update_xaxes(nticks=10)
fig.update_layout(title=dict(text="Log-scale distribution of datapoints from techs with values < 0.1", x=0.5, y=0.98),
                  showlegend=False)
fig.show()

In [8]:
# Truncate values < 0.1 to 0.1 for the following techs
tech_truncation = [
    'Telegraph Traffic',
    'Nickel Production',
    'Nuclear Energy',
    'Solar Photovoltaic'
]

data_clean.loc[data_clean['tech'].isin(tech_truncation), 'value'] = data_clean['value'].apply(lambda x: 0.1 if x < 0.1 else x)

In [9]:
data_clean

,id,country,tech,metric,unit,source,variable,year,value
838969,Steamships_Cumulative total capacity_GB,United Kingdom,Steamships,Total Number,-,CHAT,Number of Units|Steamships,1814,1.00
846327,Steamships_Cumulative total capacity_GB,United Kingdom,Steamships,Total Number,-,CHAT,Number of Units|Steamships,1815,8.00
853685,Steamships_Cumulative total capacity_GB,United Kingdom,Steamships,Total Number,-,CHAT,Number of Units|Steamships,1816,12.00
861043,Steamships_Cumulative total capacity_GB,United Kingdom,Steamships,Total Number,-,CHAT,Number of Units|Steamships,1817,14.00
868401,Steamships_Cumulative total capacity_GB,United Kingdom,Steamships,Total Number,-,CHAT,Number of Units|Steamships,1818,19.00
...,...,...,...,...,...,...,...,...,...
2373471,Onshore Wind Energy_Installed electricity capa...,Turkey,Onshore Wind Energy,Installed Capacity,MW,IRENASTAT,Installed Capacity|Electricity|Onshore Wind,2022,11396.20
2373487,Onshore Wind Energy_Installed electricity capa...,Czech Republic,Onshore Wind Energy,Installed Capacity,MW,IRENASTAT,Installed Capacity|Electricity|Onshore Wind,2022,339.41
2373491,Onshore Wind Energy_Installed electricity capa...,China,Onshore Wind Energy,Installed Capacity,MW,IRENASTAT,Installed Capacity|Electricity|Onshore Wind,2022,335504.19
2373500,Onshore Wind Energy_Installed electricity capa...,Denmark,Onshore Wind Energy,Installed Capacity,MW,IRENASTAT,Installed Capacity|Electricity|Onshore Wind,2022,4782.24


# Process AFO Data
https://alternative-fuels-observatory.ec.europa.eu/transport-mode/road/norway/vehicles-and-fleet

In [10]:
afo_ev_pass = pd.read_csv('EU_AFO_Norway_data/af_fleet_M1_2023.csv')
afo_ev_comm = pd.read_csv('EU_AFO_Norway_data/af_fleet_N1_2023.csv')

In [11]:
def process_ev_data(df, tech, id_suffix):
    return df[['Category', 'BEV', 'PHEV']].fillna(0).assign( 
        Total=lambda df: df['BEV'] + df['PHEV'],
        id=f'{tech}_{id_suffix}',
        country='Norway',
        tech=tech,
        metric='Total Number',
        unit='-',
        variable=f'Total Number|{tech}'
    ).rename(columns={'Category': 'year', 'Total': 'value'})[['id', 'country', 'tech', 'metric', 'unit', 'variable', 'year', 'value']]

ev_pass = process_ev_data(afo_ev_pass, 'Passenger EVs', 'Total Number_NOR')
ev_comm = process_ev_data(afo_ev_comm, 'LD Commercial EVs', 'Total Number_NOR')
ev_comm = ev_comm[ev_comm['value'] > 0].copy()

data_clean = pd.concat([data_clean, ev_pass, ev_comm], ignore_index=True)

# Obtain growth rates

In [12]:
def get_growth_function(function):
    if function == 'logistic':
        return lambda x, L, k, x0: L / (1 + np.exp(-k * (x - x0)))
    elif function == 'gompertz':
        return lambda x, L, k, x0: L * np.exp(-np.exp(-k * (x - x0)))
    elif function == 'softplus':
        return lambda x, L, k, x0: L / k * np.log(1 + np.exp(k * (x - x0)))     # from https://doi.org/10.1038/s43247-023-01056-1 - relatively untested
    
def growth_rate_fitting(data=data_clean, error_metric='mae', country=None, tech=None, normalize=False, sort_by='rate', country_rate=None, rate_cuttoff=None, r2_cuttoff=None, mape_cuttoff=None,
                        formulations = ['logistic', 
                                        'gompertz', 
                                        # 'softplus'                            # yielding very high growth rates
                                        ]                                               
                        ):
    if error_metric not in ['rmse', 'mae', 'mape']:
        raise ValueError(f"Unsupported error metric: {error_metric}")
    if sort_by not in ['rate', 'r2']:
        raise ValueError(f"Unsupported sorting param: {sort_by}")
    if r2_cuttoff is not None and (not isinstance(r2_cuttoff, (float, int)) or not (0 <= r2_cuttoff <= 1)):
        raise ValueError("r2_cuttoff must be a number between 0 and 1.")

    growth_rates = data[['tech', 'country', 'metric', 'unit']].drop_duplicates().reset_index(drop=True)
    growth_rates = growth_rates.assign(rate=np.nan, asymptote=np.nan, inflection=np.nan, 
                                       r2=np.nan, rmse=np.nan, mae=np.nan, formulation=pd.Series(dtype='string'))
    k_upper = 3.0                                                               # k upper bound is set to 3 as per the upper-range Odenweller et al. 2022 emergency deployment growth rates

    if country is not None:
        growth_rates = growth_rates[growth_rates['country'] == country]

    if tech is not None:
        growth_rates = growth_rates[growth_rates['tech'] == tech]

    for i, row in growth_rates.iterrows():
        tech = row['tech']
        country = row['country']
        hist_growth = data[(data['tech'] == tech) & (data['country'] == country)]
        x = hist_growth['year']
        y = hist_growth['value']
        if normalize:                                                          # normalize the data
            y = (y - y.min()) / (y.max() - y.min())

        best_metric, best_params, best_formulation, best_r2 = float('inf'), None, None, None
        for func in formulations:
            f = get_growth_function(func)
            L0 = y.max()                                                       # initial guesses
            k0 = 0.1
            x0 = np.median(x) if func in ['logistic', 'softplus'] else x.min()

            try:
                popt, _ = curve_fit(f, x.values, y.values, [L0, k0, x0], 
                                    # maxfev=17000                             # does not converge in less iterations for some techs
                                    maxfev=10000,
                                    bounds=([L0*0.9, 0, x.min()], [L0*1.1, k_upper, x.max()])     
                                    )
                
                y_pred = f(x.values, *popt)                
                # y_pred = savgol_filter(y_pred, 9, 2)                         # smoothing technique
                ss_res = np.sum((y.values - y_pred) ** 2)
                ss_tot = np.sum((y.values - y.values.mean()) ** 2)
                r2 = 1 - (ss_res / ss_tot)
                rmse = np.sqrt(ss_res / len(y))                                # len(y), not len(y) - 1
                mae = np.mean(np.abs(y.values - y_pred))
                mape = np.mean(np.abs((y.values - y_pred)/y.values))

                error_dict = {'rmse': rmse, 'mae': mae, 'mape': mape}
                current_metric = error_dict[error_metric]   
                if current_metric < best_metric:                               # check for formulation with the lowest error_metric
                    best_metric, best_params, best_formulation, best_r2, best_mape = current_metric, popt, func, r2, mape

            except RuntimeError:
                print(f"Optimal {func} params not found in {tech} from {country}, removed row")    
                continue        # if nan, will be removed

        # Update the DataFrame with the best formulation's results
        if best_params is not None:
            growth_rates.at[i, 'rate'] = best_params[1]                 # k, target parameter
            growth_rates.at[i, 'asymptote'] = best_params[0]            # L, for demonstration
            growth_rates.at[i, 'inflection'] = best_params[2]           # x0, for demonstration
            growth_rates.at[i, 'r2'] = best_r2                          # Coefficient of determination
            growth_rates.at[i, 'mape'] = best_mape                      # Mean absolute percentage error
            growth_rates.at[i, 'rmse'] = rmse                           # Root mean squared error - penalizes large deviations more (sensitive to outliers)
            growth_rates.at[i, 'mae'] = mae                             # Mean absolute error
            growth_rates.at[i, 'formulation'] = best_formulation        # Formulation with the lowest error_metric

    growth_rates = growth_rates[growth_rates['rate'] >= 0]
    growth_rates = growth_rates[round(growth_rates['rate'], 3) < k_upper]       # Remove techs artificially truncated to the upper bound

    if rate_cuttoff is not None:
            growth_rates = growth_rates[growth_rates['rate'] < rate_cuttoff]    # Apply rate_cuttoff filter
    if r2_cuttoff is not None:
            growth_rates = growth_rates[growth_rates['r2'] > r2_cuttoff]        # Apply r2_cuttoff filter
    if mape_cuttoff is not None:
            growth_rates = growth_rates[growth_rates['mape'] < mape_cuttoff]    # Apply mape_cuttoff filter
    if country_rate is not None:
        if country_rate == 'max':
            max_rate_i = growth_rates.groupby('tech')['rate'].idxmax()          # Select country with max growth rate
            growth_rates = growth_rates.loc[max_rate_i]
        elif country_rate == 'median':
            median_rate_i = growth_rates.groupby('tech')['rate'].apply(lambda x: x.idxmin() if len(x) == 1 else x.sub(x.median()).abs().idxmin())   # x - x.median() to find the minimum difference, being the closest to the median
            growth_rates = growth_rates.loc[median_rate_i]
        else: print('Select "max" or "median" country-specific rates')
        return growth_rates.sort_values(sort_by, ascending=False).reset_index(drop=True)
    return growth_rates.sort_values(sort_by, ascending=False).reset_index(drop=True)

In [13]:
def plot_growth_rates(data=data_clean, error_metric='mae', specific_tech=None, specific_country=None, normalize=False, sort_by='rate', country_rate=None, rate_cuttoff=None, r2_cuttoff=None, mape_cuttoff=None,
                      formulations = ['logistic', 
                                    'gompertz', 
                                    # 'softplus'                            # yielding very high growth rates
                                    ]                                               
                      ):
    if specific_tech is None and specific_country is None and country_rate is None:
        print("Too many plots to show. Define a specific technology, a specific country, or to show selected rates per country.")
        return
    
    rates = growth_rate_fitting(data=data, error_metric=error_metric, tech=specific_tech, country=specific_country, normalize=normalize, sort_by=sort_by, 
                                country_rate=country_rate, rate_cuttoff=rate_cuttoff, r2_cuttoff=r2_cuttoff, mape_cuttoff=mape_cuttoff, formulations=formulations)
    num_rows = len(rates)
    if num_rows == 0:
        print("No data available for plotting with the specified filters.")
        return
    cols = 4 if num_rows >= 4 else num_rows
    rows = -(-num_rows // cols)  # Ceiling division for rows

    fig = make_subplots(rows=rows, cols=cols, 
                        vertical_spacing=0.22/rows, 
                        horizontal_spacing=0.05,
                        subplot_titles=[f"<b>{row['tech']}</b> in {row['country']}<br>({row['formulation']}'s growth rate={row['rate']:.3f})" for _, row in rates.iterrows()])
    annotations = []
    for i, row in rates.iterrows():
        tech, country, formulation = row['tech'], row['country'], row['formulation']
        f = get_growth_function(formulation)
        
        L, k, x0 = row['asymptote'], row['rate'], row['inflection']
        hist = data[(data['tech'] == tech) & (data['country'] == country)]
        y, x = hist['value'].values, hist['year'].values 
        if normalize: y = (y - y.min()) / (y.max() - y.min())
        x_fit = np.linspace(x.min(), x.max(), 100)       
        y_fit = f(x_fit, L, k, x0)

        row_i = (i // cols) + 1
        col_i = (i % cols) + 1

        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name=f"{tech} - {country}",
                                 marker=dict(size=4, opacity=0.8, color='black')),
                      row=row_i, col=col_i)
        fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode='lines', name=f"{tech} - {country} Curve Fit",
                                 line=dict(width=2.5, color='red')),
                      row=row_i, col=col_i)
        fig.update_yaxes(title_text=f"{row['metric']} [{row['unit']}]", row=row_i, col=col_i, title_standoff=5)
        annotations.append(
            dict(
                xref=f"x{(i+1)}",  # Use subplot-specific axis
                yref=f"y{(i+1)}",
                x=x.min(),  # Place at the minimum x-value
                y=y.max(),  # Place near the maximum y-value
                text=f"R²: {row['r2']:.3f}<br>MAPE: {row['mape']:.3f}",
                showarrow=False,
                xanchor="left",
                yanchor="top",
            )
        )
    existing_annotations = list(fig['layout']['annotations'])
    fig.update_layout(
        annotations=existing_annotations + annotations
    )

    if country_rate is not None: title_text = f'{country_rate.capitalize()} technology adoption growth rates of selected countries; N = {num_rows} techs'
    elif specific_tech is not None: title_text = f'Historical adoption growth rates of {specific_tech} from {num_rows} countries'
    else: title_text = f'Historical technology adoption growth rates from {num_rows} techs in {specific_country}'
    
    if normalize: title_text += ' (normalized)'
    if rate_cuttoff is not None: title_text += f' (growth rate < {rate_cuttoff})'
    if r2_cuttoff is not None: title_text += f' (where R² > {r2_cuttoff})'        

    
    fig.update_layout(
        title=dict(text=title_text, x=0.5, y=0.995, font=dict(size=18)),
        font=dict(size=11),
        showlegend=False,
        height=250 * rows,
        width=350 * cols,
        template='plotly_white',
        annotations=[
        dict(font=dict(size=12)) for annotation in fig['layout']['annotations']
        ]
    )
    fig.show()

In [14]:
plot_growth_rates(
    data=data_clean[data_clean['country'] != 'World'],
    # specific_tech='Solar Photovoltaic', 
    # specific_country='Canada',
    # normalize=True,
    # sort_by='r2',
    country_rate='max',
    rate_cuttoff=2.0,
    r2_cuttoff=0.8,
    # mape_cuttoff=0.2,
    formulations=['logistic']
    )

C:\Users\rashi\AppData\Local\Temp\ipykernel_35704\67593570.py:60: RuntimeWarning:

divide by zero encountered in scalar divide



In [15]:
k_rates = growth_rate_fitting(
    data=data_clean[data_clean['country'] != 'World'],
    # tech='Solar Photovoltaic', 
    # country='Canada',
    # normalize=True,
    # sort_by='r2',
    # country_rate='median',
    # rate_cuttoff=2.0,
    r2_cuttoff=0.8,
    # mape_cuttoff=0.2
    )

C:\Users\rashi\AppData\Local\Temp\ipykernel_35704\67593570.py:60: RuntimeWarning:

divide by zero encountered in scalar divide



In [16]:
k_rates

,tech,country,metric,unit,rate,asymptote,inflection,r2,rmse,mae,formulation,mape
0,Nickel Production,South Africa,Annual Production,thousand metric tons,2.886087,3.675780e+04,1993.454603,0.992668,1.178394e+03,3.846491e+02,gompertz,1.299286
1,Solid Biofuels,Korea,Installed Capacity,MW,2.880260,1.848380e+03,2015.803219,0.994275,6.090139e+01,4.402746e+01,logistic,0.686028
2,Solar Photovoltaic,Czech Republic,Installed Capacity,MW,2.755241,2.066386e+03,2009.439168,0.998860,3.029841e+01,1.685522e+01,logistic,0.335743
3,Sulphuric Acid,Indonesia,Annual Production,thousand metric tons,2.561220,1.079100e+03,1998.082247,0.915733,5.638225e+01,3.704027e+01,gompertz,0.920000
4,Onshore Wind Energy,Russia,Installed Capacity,MW,2.375085,2.231653e+03,2020.141702,0.999380,2.041559e+01,1.537328e+01,logistic,0.826207
...,...,...,...,...,...,...,...,...,...,...,...,...
506,Hydroelectricity,Hungary,Annual Production,TWh,0.035710,2.709000e-01,1968.701357,0.815686,2.083551e-02,1.588765e-02,gompertz,0.099759
507,Sand and Gravel|Construction,United States,Annual Production,metric tons,0.032822,1.206000e+09,1953.034257,0.900758,1.198667e+08,9.492830e+07,gompertz,0.427203
508,Zinc,Australia,Annual Production,thousand metric tons,0.026698,6.292000e+02,1971.587398,0.934636,3.544532e+01,2.183877e+01,gompertz,0.464291
509,Crude Oil,United States,Annual Production,thousand metric tons,0.022829,5.887710e+05,1944.812497,0.823121,7.105825e+04,4.737170e+04,gompertz,1.009451


In [17]:
fig = px.violin(k_rates, y='rate', box=True, points='all', 
                template='plotly_white', width=500, height=500,
                hover_data=['tech', 'country', 'rate'])

fig.show()

In [18]:
fig = px.box(k_rates, x='rate', y='tech',
            template='plotly_white', width=1200, height=1200,
            hover_data=['tech', 'country', 'rate'])
fig.show()

In [19]:
k_selected = growth_rate_fitting(
    data=data_clean[data_clean['country'] != 'World'],
    # tech='Solar Photovoltaic', 
    # country='Canada',
    # normalize=True,
    # sort_by='r2',
    country_rate='max',
    rate_cuttoff=2.0,
    r2_cuttoff=0.8,
    # mape_cuttoff=0.2
    # formulations=['logistic']
    )

C:\Users\rashi\AppData\Local\Temp\ipykernel_35704\67593570.py:60: RuntimeWarning:

divide by zero encountered in scalar divide



In [20]:
k_percentiles = k_selected['rate'].quantile([0., 0.1, 0.25, 0.5, 0.75, 0.9, 1.], interpolation='nearest')
k_selected.loc[k_selected['rate'].isin(k_percentiles.values)].assign(percentile=k_percentiles.index[::-1])[['percentile', 'tech', 'country', 'rate', 'metric']]

,percentile,tech,country,rate,metric
0,1.00,Biogas,Mexico,1.220175,Installed Capacity
5,0.90,Geothermal Energy,Russia,0.976849,Installed Capacity
14,0.75,Wet Flue Gas Desulfurization Systems,Germany,0.533747,Cumulative Total Capacity
27,0.50,Zinc,Japan,0.290746,Annual Production
40,0.25,Natural Gas Production,Indonesia,0.187163,Annual Production
49,0.10,Refrigerators,United States,0.072338,Cumulative Total Capacity
54,0.00,Public Roads,United States,0.020486,Cumulative Length


In [21]:
k_selected[(k_selected['tech'].str.contains('Car')) | (k_selected['tech'].str.contains('EV'))][['tech', 'rate']]

,tech,rate
10,LD Commercial EVs,0.697463
15,Passenger EVs,0.529813
22,Passenger Cars,0.378716


# Obtain saturation level and initial capacities

In [22]:
def map_tech_to_class(tech):
    if tech.startswith("T_LDV_C_"):
        return "Cars"
    elif tech.startswith("T_LDV_LTP"):
        return "Pass Light-T"
    elif tech.startswith("T_LDV_LTF"):
        return "Freight Light-T"
    elif tech.startswith("T_MDV_T"):
        return "Medium-T"
    elif tech.startswith("T_HDV_T"):
        return "Heavy-T"
    else: return None
    
def map_tech_to_drivetrain(tech):
    if 'BEV' in tech:
        return 'Battery'
    if 'PHEV' in tech:
        return 'Plug-in Hybrid'
    if 'HEV' in tech:
        return 'Hybrid'
    if 'FCEV' in tech:
        return 'Fuel-cell'
    if 'CNG' in tech:
        return 'Natural gas'
    if 'DSL_N' in tech:
        return 'Diesel'
    else: return None

def import_vanilla_data(results_dir='../results_analysis/vanilla4/vanilla4.xlsx',
                        excel_dir = '../transportation/spreadsheet_database/CANOE_TRN_ON_v4.xlsx', new2020_sheet='ExCap',
                        netcap_sheet='Capacity_Transport', newcap_sheet='NewCapacity_Transport', 
                        c0_net_ratio=0.001, c0_new_ratio=0.001):
    """
    Imports CANOE-vanilla capacity outputs driven by demand projections from base scenario
    """
    # Saturation levels for net and new capacities
    net = pd.read_excel(results_dir, sheet_name=netcap_sheet).drop('Region', axis=1).rename(columns={'Technology': 'tech'}).fillna(0)
    net = net.melt(id_vars=['tech'], value_vars=[2021, 2025, 2030, 2035, 2040, 2045, 2050], var_name='year', value_name='value')
    net['class'] = net['tech'].apply(map_tech_to_class)
    net = net[net['class'].notnull()].reset_index(drop=True).copy()
    net_year = net.groupby(['class', 'year'], as_index=False)['value'].sum()      # fleet size per class per year
    net_max = net_year.groupby('class', as_index=False)['value'].max()            # fleet saturation per class

    new = pd.read_excel(results_dir, sheet_name=newcap_sheet).drop('Region', axis=1).rename(columns={'Technology': 'tech'}).fillna(0)
    new = new.melt(id_vars=['tech'], value_vars=[2021, 2025, 2030, 2035, 2040, 2045, 2050], var_name='year', value_name='value')
    new['class'] = new['tech'].apply(map_tech_to_class)
    new = new[new['class'].notnull()].reset_index(drop=True).copy()
    new_year = new.groupby(['class', 'year'], as_index=False)['value'].sum()      # market size per class per year
    new_max = new_year.groupby('class', as_index=False)['value'].max()            # market saturation per class (also used as market pull proxy)
    # new_max['value'] /= 5                                                         # divide by 5 to get avg. annual sales - TODO: check if this is correct; Temoa uses 5 years for market pull
    new_max['max_pull'] = np.ceil(new_max['value'])

    cap_sat = pd.merge(net_max, new_max, on='class', suffixes=('_net', '_new'))
    cap_sat.rename(columns={'value_net': 'L_net', 'value_new': 'L_new'}, inplace=True)      # saturation level params    
    
    # Initial net and new capacities
    net_2020 = net[                                                               # initial net capacity (sum of existing capacities by tech)
        (net['tech'].str.contains('EX')) & (net['year'] == 2021)
        ].reset_index(drop=True).drop('year', axis=1).copy()
    net_2020.loc[:, 'tech'] = net_2020['tech'].str.replace('EX', 'N')
    net_2020['class'] = net_2020['tech'].apply(map_tech_to_class)
                                                                                  # initial new capacity (max historical sales by tech)
    new_2020 = pd.read_excel(excel_dir, sheet_name=new2020_sheet, skiprows=[0], usecols=range(26)).drop(['Reference', 'Region', 'Data Year', 'Unit'], axis=1).fillna(0)
    new_2020.rename(columns={'Technology': 'tech'}, inplace=True)
    new_2020 = new_2020[new_2020['tech'] != 0]
    vintages = [col for col in new_2020.columns if str(col).isdigit()]
    new_2020 = new_2020.melt(id_vars=['tech'], value_vars=vintages, var_name='year', value_name='value')
    new_2020 = new_2020.groupby('tech', as_index=False).max()
    new_2020['class'] = new_2020['tech'].apply(map_tech_to_class)
    new_2020 = new_2020[new_2020['class'].notnull()].reset_index(drop=True).drop('year', axis=1).copy()
    new_2020.loc[:, 'tech'] = new_2020['tech'].str.replace('EX', 'N')

    newtech = net[(net['year'] == 2021) & ~(net['tech'].str.contains('EX'))]     # to account for techs with no initial 2020 capacity 
    newtech.loc[:, 'value'] = 0 
    newtech = newtech[                                                           # filter duplicate techs (including AER variations)
        ~(newtech['tech'].isin(net_2020['tech']))
        & ~(newtech['tech'].str.contains('BEV200'))
        & ~(newtech['tech'].str.contains('BEV400'))
        & ~(newtech['tech'].str.contains('PHEV50'))
        & ~(newtech['tech'].str.contains('FCHEV'))
        & (newtech['tech'] != 'T_LDV_C_BEV300_N')
        & (newtech['tech'] != 'T_LDV_LTP_BEV150_N')                                
        & (newtech['tech'] != 'T_LDV_LTF_BEV150_N')                                                               
        ].reset_index(drop=True).drop('year', axis=1).copy()
    newtech['class'] = newtech['tech'].apply(map_tech_to_class)

    net_2020 = pd.concat([net_2020, newtech], ignore_index=True).sort_values(['class', 'value'], ascending=[True, False]).reset_index(drop=True)
    net_year_dict = net_year[net_year['year'] == 2021].set_index('class')['value'].to_dict()
    net_2020.loc[:, 'value'] = net_2020.apply(lambda row: net_year_dict[row['class']] * c0_net_ratio if row['value'] < net_year_dict[row['class']] * c0_net_ratio else row['value'], axis=1)

    new_2020 = pd.concat([new_2020, newtech], ignore_index=True).sort_values(['class', 'value'], ascending=[True, False]).reset_index(drop=True)
    new_year_dict = new_year[new_year['year'] == 2021].set_index('class')['value'].to_dict()
    new_2020.loc[:, 'value'] = new_2020.apply(lambda row: new_year_dict[row['class']] * c0_new_ratio if row['value'] < new_year_dict[row['class']] * c0_new_ratio else row['value'], axis=1)

    cap_0 = pd.merge(net_2020, new_2020, on=['tech', 'class'], suffixes=('_net', '_new'))    # filter out conventional techs
    cap_0 = cap_0[
        ~(cap_0['tech'].str.contains('GSL_N'))
        & (cap_0['tech'] != 'T_LDV_LTP_DSL_N')
        & (cap_0['tech'] != 'T_LDV_LTF_DSL_N')
        & (cap_0['tech'] != 'T_MDV_T_DSL_N')
        & (cap_0['tech'] != 'T_HDV_T_DSL_N')
    ].reset_index(drop=True)
    cap_0.rename(columns={'value_net': 'c0_net', 'value_new': 'c0_new'}, inplace=True)
    cap_0['powertrain'] = cap_0['tech'].apply(map_tech_to_drivetrain)
    cap_params = pd.merge(cap_sat, cap_0, on='class')[['tech', 'class', 'powertrain', 'c0_net', 'c0_new', 'L_net', 'L_new', 'max_pull']]

    return cap_params
    
import_vanilla_data()

c:\Users\rashi\miniconda3\envs\main\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning:

Data Validation extension is not supported and will be removed



,tech,class,powertrain,c0_net,c0_new,L_net,L_new,max_pull
0,T_LDV_C_GSL_HEV_N,Cars,Hybrid,78.217394,13.694424,7510.985907,3071.729696,3072.0
1,T_LDV_C_DSL_N,Cars,Diesel,62.314640,9.876340,7510.985907,3071.729696,3072.0
2,T_LDV_C_BEV150_N,Cars,Battery,40.816915,11.479291,7510.985907,3071.729696,3072.0
3,T_LDV_C_GSL_PHEV35_N,Cars,Plug-in Hybrid,25.422725,8.662460,7510.985907,3071.729696,3072.0
4,T_LDV_C_CNG_N,Cars,Natural gas,4.272709,0.457183,7510.985907,3071.729696,3072.0
5,T_LDV_C_FCEV400_N,Cars,Fuel-cell,4.272709,0.457183,7510.985907,3071.729696,3072.0
6,T_LDV_LTF_GSL_PHEV35_N,Freight Light-T,Plug-in Hybrid,1.149195,0.494326,2286.493550,694.061817,695.0
7,T_LDV_LTF_GSL_HEV_N,Freight Light-T,Hybrid,1.149195,0.114755,2286.493550,694.061817,695.0
8,T_LDV_LTF_CNG_N,Freight Light-T,Natural gas,1.149195,0.114755,2286.493550,694.061817,695.0
9,T_LDV_LTF_BEV300_N,Freight Light-T,Battery,1.149195,0.114755,2286.493550,694.061817,695.0


In [23]:
def import_ref2021_data(net_results_dir='../results_analysis/ref2021_net/ref2_freegrowth_net.xlsx',
                        new_results_dir='../results_analysis/ref2021_new/ref2_freegrowth_new.xlsx',
                        netcap_sheet='Capacity_Transport', newcap_sheet='NewCapacity_Transport', 
                        c0_net_ratio=0.001, c0_new_ratio=0.001):
    """
    Imports CANOE-vanilla capacity outputs driven by demand projections from base scenario
    """
    # Saturation levels for net and new capacities
    net = pd.read_excel(net_results_dir, sheet_name=netcap_sheet).drop('Region', axis=1).rename(columns={'Technology': 'tech'}).fillna(0)
    net = net.melt(id_vars=['tech'], value_vars=[2021, 2025, 2030, 2035, 2040, 2045, 2050], var_name='year', value_name='value')
    net['class'] = net['tech'].apply(map_tech_to_class)
    net = net[net['class'].notnull()].reset_index(drop=True).copy()
    net_year = net.groupby(['class', 'year'], as_index=False)['value'].sum()      # fleet size per class per year
    net_max = net_year.groupby('class', as_index=False)['value'].max()            # fleet saturation per class

    new = pd.read_excel(new_results_dir, sheet_name=newcap_sheet).drop('Region', axis=1).rename(columns={'Technology': 'tech'}).fillna(0)
    new = new.melt(id_vars=['tech'], value_vars=[2021, 2025, 2030, 2035, 2040, 2045, 2050], var_name='year', value_name='value')
    new['class'] = new['tech'].apply(map_tech_to_class)
    new = new[new['class'].notnull()].reset_index(drop=True).copy()
    new_year = new.groupby(['class', 'year'], as_index=False)['value'].sum()      # market size per class per year
    new_max = new_year.groupby('class', as_index=False)['value'].max()            # market saturation per class (also used as market pull proxy)
    # new_max['value'] /= 5                                                         # divide by 5 to get avg. annual sales - TODO: check if this is correct; Temoa uses 5 years for market pull
    new_max['max_pull'] = np.ceil(new_max['value'])

    cap_sat = pd.merge(net_max, new_max, on='class', suffixes=('_net', '_new'))
    cap_sat.rename(columns={'value_net': 'L_net', 'value_new': 'L_new'}, inplace=True)      # saturation level params    
    
    # Initial net and new capacities
    net_2021 = net[net['year'] == 2021].reset_index(drop=True).drop('year', axis=1).copy()      # initial net capacity (sum of existing and 2021 capacities by tech)
    net_2021.loc[:, 'tech'] = net_2021['tech'].str.replace('EX', 'N')
    net_2021 = net_2021.groupby('tech', as_index=False).sum()
    net_2021['class'] = net_2021['tech'].apply(map_tech_to_class)

    new_2021 = new[new['year'] == 2021].reset_index(drop=True).drop('year', axis=1).copy()      # initial new capacity (2021 capacities by tech)
    new_2021 = new_2021[~new_2021['tech'].str.contains('EX')].reset_index(drop=True)
    new_2021['class'] = new_2021['tech'].apply(map_tech_to_class)

    tech_query = (
    (net_2021['tech'] != 'T_LDV_C_BEV300_N') &
    (net_2021['tech'] != 'T_LDV_LTP_BEV150_N') &
    (net_2021['tech'] != 'T_LDV_LTF_BEV150_N') &
    (~net_2021['tech'].str.contains('BEV200')) &
    (~net_2021['tech'].str.contains('BEV400')) &
    (~net_2021['tech'].str.contains('PHEV50')) &
    (~net_2021['tech'].str.contains('FCHEV'))
    )
    net_2021 = net_2021[tech_query].sort_values(['class', 'value'], ascending=[True, False]).reset_index(drop=True)
    new_2021 = new_2021[tech_query]
    new_2021 = pd.concat([new_2021, net_2021[~net_2021['tech'].isin(new_2021['tech'])]], ignore_index=True).sort_values(['class', 'value'], ascending=[True, False]).reset_index(drop=True)

    net_year_dict = net_year[net_year['year'] == 2021].set_index('class')['value'].to_dict()
    net_2021.loc[:, 'value'] = net_2021.apply(lambda row: net_year_dict[row['class']] * c0_net_ratio if row['value'] < net_year_dict[row['class']] * c0_net_ratio else row['value'], axis=1)
    new_year_dict = new_year[new_year['year'] == 2021].set_index('class')['value'].to_dict()
    new_2021.loc[:, 'value'] = new_2021.apply(lambda row: new_year_dict[row['class']] * c0_new_ratio if row['value'] < new_year_dict[row['class']] * c0_new_ratio else row['value'], axis=1)

    cap_0 = pd.merge(net_2021, new_2021, on=['tech', 'class'], suffixes=('_net', '_new'))    # filter out conventional techs
    cap_0 = cap_0[
        ~(cap_0['tech'].str.contains('GSL_N'))
        & (cap_0['tech'] != 'T_LDV_LTP_DSL_N')
        & (cap_0['tech'] != 'T_LDV_LTF_DSL_N')
        & (cap_0['tech'] != 'T_MDV_T_DSL_N')
        & (cap_0['tech'] != 'T_HDV_T_DSL_N')
    ].reset_index(drop=True)
    cap_0.rename(columns={'value_net': 'c0_net', 'value_new': 'c0_new'}, inplace=True)
    cap_0['powertrain'] = cap_0['tech'].apply(map_tech_to_drivetrain)
    cap_params = pd.merge(cap_sat, cap_0, on='class')[['tech', 'class', 'powertrain', 'c0_net', 'c0_new', 'L_net', 'L_new', 'max_pull']]

    return cap_params

import_ref2021_data()

,tech,class,powertrain,c0_net,c0_new,L_net,L_new,max_pull
0,T_LDV_C_GSL_HEV_N,Cars,Hybrid,96.504725,43.386708,7514.810859,3069.113127,3070.0
1,T_LDV_C_DSL_N,Cars,Diesel,62.314640,0.457183,7514.810859,3069.113127,3070.0
2,T_LDV_C_BEV150_N,Cars,Battery,59.087370,42.022888,7514.810859,3069.113127,3070.0
3,T_LDV_C_GSL_PHEV35_N,Cars,Plug-in Hybrid,25.422725,3.427794,7514.810859,3069.113127,3070.0
4,T_LDV_C_CNG_N,Cars,Natural gas,4.272709,0.457183,7514.810859,3069.113127,3070.0
5,T_LDV_C_FCEV400_N,Cars,Fuel-cell,4.272709,0.457183,7514.810859,3069.113127,3070.0
6,T_LDV_LTF_GSL_HEV_N,Freight Light-T,Hybrid,9.246201,4.831168,2282.233573,689.652523,690.0
7,T_LDV_LTF_GSL_PHEV35_N,Freight Light-T,Plug-in Hybrid,1.468433,0.455923,2282.233573,689.652523,690.0
8,T_LDV_LTF_BEV300_N,Freight Light-T,Battery,1.149195,0.114755,2282.233573,689.652523,690.0
9,T_LDV_LTF_CNG_N,Freight Light-T,Natural gas,1.149195,0.114755,2282.233573,689.652523,690.0


In [24]:
cap_params = import_ref2021_data(
    c0_net_ratio=0.001, c0_new_ratio=0.001,                        # assumption: percentage of 2021 fleet and market size to use as initial capacities
)
k_scenarios = pd.DataFrame({
    'scenario': ['25th percentile', 'Median', '90th percentile', 'Norway EVs'],
    'rate': [k_percentiles[0.25], k_percentiles[0.5], k_percentiles[0.9], k_selected[k_selected['tech'] == 'Passenger EVs']['rate'].values[0]]
})
k_scenarios

,scenario,rate
0,25th percentile,0.187163
1,Median,0.290746
2,90th percentile,0.976849
3,Norway EVs,0.529813


In [25]:
growth_params = cap_params.merge(k_scenarios, how='cross')
growth_params

,tech,class,powertrain,c0_net,c0_new,L_net,L_new,max_pull,scenario,rate
0,T_LDV_C_GSL_HEV_N,Cars,Hybrid,96.504725,43.386708,7514.810859,3069.113127,3070.0,25th percentile,0.187163
1,T_LDV_C_GSL_HEV_N,Cars,Hybrid,96.504725,43.386708,7514.810859,3069.113127,3070.0,Median,0.290746
2,T_LDV_C_GSL_HEV_N,Cars,Hybrid,96.504725,43.386708,7514.810859,3069.113127,3070.0,90th percentile,0.976849
3,T_LDV_C_GSL_HEV_N,Cars,Hybrid,96.504725,43.386708,7514.810859,3069.113127,3070.0,Norway EVs,0.529813
4,T_LDV_C_DSL_N,Cars,Diesel,62.314640,0.457183,7514.810859,3069.113127,3070.0,25th percentile,0.187163
...,...,...,...,...,...,...,...,...,...,...
91,T_LDV_LTP_CNG_N,Pass Light-T,Natural gas,4.112463,0.301148,8112.435884,2393.973010,2394.0,Norway EVs,0.529813
92,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,4.112463,0.301148,8112.435884,2393.973010,2394.0,25th percentile,0.187163
93,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,4.112463,0.301148,8112.435884,2393.973010,2394.0,Median,0.290746
94,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,4.112463,0.301148,8112.435884,2393.973010,2394.0,90th percentile,0.976849


# Simulate deployment curves

In [26]:
def growth_substitution(model):
    if model == 'logistic':
        return lambda i, C, k, L: C[i-1] * (1 + k * (1 - C[i-1]/L))
    elif model == 'gompertz':
        return lambda i, C, k, L: C[i-1] * (1 + k * np.log(L/C[i-1]))
    elif model == 'softplus':
        return lambda i, C, k, L: C[i-1] + L * (1 - np.exp(-k/L * C[i-1]))

def deployment_curves(params=growth_params, p0=2021, pf=2050, model='logistic', capacity_type='net', market_pull_restrict=True, precision=9):
    periods = np.arange(p0, pf + 1)

    params = params[['tech', 'class', 'powertrain', 'max_pull', 'c0_' + capacity_type, 'L_' + capacity_type, 'scenario', 'rate']]       # filters the unwanted capacity params
    data = params.loc[params.index.repeat(len(periods))].assign(period=np.tile(periods, len(params)), capacity=np.nan).reset_index(drop=True)

    for i, row in params.iterrows():
        tech, scenario, c0, L, k, pull = row['tech'], row['scenario'], row['c0_' + capacity_type], row['L_' + capacity_type], row['rate'], row['max_pull']
        f = growth_substitution(model)
        x = periods
        C = np.zeros(len(x))
        C[0] = c0

        for i in range(1, len(x)):
            if market_pull_restrict:                                        # limits capacity growth to the yearly market pull by class
                y = f(i, C, k, L)
                y_prev = f(i-1, C, k, L)
                C[i] = y if y-y_prev <= pull else y_prev + pull
            else: C[i] = f(i, C, k, L)
        
        data.loc[(data['tech'] == tech) & (data['scenario'] == scenario), 'capacity'] = C        
        
    data['capacity'], data['rate'] = round(data['capacity'], precision), round(data['rate'], precision)
    data['c0_' + capacity_type], data['L_' + capacity_type] = round(data['c0_' + capacity_type], precision), round(data['L_' + capacity_type], precision)
    return data

In [27]:
def plot_deployment_curves(params=growth_params, p0=2021, pf=2050, model='logistic', capacity_type='net', market_pull_restrict=True):
    df = deployment_curves(params, p0, pf, model, capacity_type, market_pull_restrict)

    fig = px.line(df, x='period', y='capacity', color='scenario', facet_row='class', facet_col='powertrain',
                template='plotly_white', height=1000, width=1500, facet_col_spacing=0.03, facet_row_spacing=0.02,
                hover_data=['tech', 'scenario', 'period', 'capacity', 'c0_' + capacity_type, 'L_' + capacity_type, 'rate'],     # order is used in c0 and L annotations
                category_orders={'powertrain': ['Battery', 'Hybrid', 'Plug-in Hybrid', 'Fuel-cell', 'Natural gas', 'Diesel'],
                                'class': ['Cars', 'Pass Light-T', 'Freight Light-T', 'Medium-T', 'Heavy-T']})

    title_text = f"Vehicle technology adoption ({capacity_type} capacity) by class and powertrain"
    fig.update_layout(
        title=dict(text=title_text, x=0.5, y=0.99, font=dict(size=18)),
        legend=dict(
            orientation='h',  
            yanchor='top',
            y=1.06,  
            xanchor='center',
            x=0.5),
        font=dict(
            size=14),
        margin=dict(
            t=85, b=0, r=10),
        )

    fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
    fig.for_each_annotation(lambda a: a.update(font=dict(size=14)))
    fig.for_each_yaxis(lambda axis: axis.title.update(text=''))
    fig.for_each_xaxis(lambda axis: axis.title.update(text=''))

    fig.update_xaxes(tickangle=45, tickfont=dict(size=12))
    fig.update_yaxes(matches=None, tickfont=dict(size=12))

    for trace in fig.data:
        if trace['name'] == '90th percentile':      # to avoid repeated annotations
            c0_value = trace['customdata'][0][2]    # value index based on hover_data order
            xref = trace['xaxis']
            yref = trace['yaxis']
            fig.add_annotation(
                x=trace['x'][0], y=trace['y'][-1], 
                text=f"c0 = {c0_value:.3f}",
                showarrow=False, 
                xref=xref, yref=yref,
                xanchor="left", yanchor="top", 
                font=dict(size=12),
            )

    fig.add_annotation(
        text=f"{capacity_type.capitalize()} capacity (k vehicles)",
        x=-0.05, 
        y=0.5,
        xref="paper",
        yref="paper",
        showarrow=False,
        textangle=-90,
        font=dict(size=16))

    fig.show()

In [28]:
plot_deployment_curves(
    model='logistic',
    capacity_type='net',
    market_pull_restrict=True
)

In [40]:
deployment_data = deployment_curves(
    capacity_type='net',
    market_pull_restrict=True
)
# deployment_data[deployment_data['scenario'] == 'Norway EVs'].to_clipboard(index=False)
deployment_data

,tech,class,powertrain,max_pull,c0_net,L_net,scenario,rate,period,capacity
0,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2021,96.504725
1,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2022,114.334920
2,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2023,135.408645
3,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2024,160.295518
4,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2025,189.657015
...,...,...,...,...,...,...,...,...,...,...
2875,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,Norway EVs,0.529813,2046,8066.994360
2876,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,Norway EVs,0.529813,2047,8090.935018
2877,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,Norway EVs,0.529813,2048,8102.296267
2878,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,Norway EVs,0.529813,2049,8107.661655


# Export growth constraints

In [41]:
def map_tech_to_group(tech):
    if tech.endswith("HEV_N"):
        return "Single"
    elif tech.endswith("DSL_N"):
        return "Single"
    elif tech.endswith("CNG_N"):
        return "Single"
    elif tech.endswith("PHEV_N"):
        return "Single"
    elif tech.endswith("BEV_N"):
        return "Single"
    elif tech.endswith("FCEV400_N"):
        return "Single"
    elif "C_GSL_PHEV" in tech:
        return "Cars PHEV"
    elif "LTP_GSL_PHEV" in tech:
        return "Pass LT PHEV"
    elif "LTF_GSL_PHEV" in tech:
        return "Freight LT PHEV"
    elif "C_BEV" in tech:
        return "Cars BEV"
    elif "LTP_BEV" in tech:
        return "Pass LT BEV"
    elif "LTF_BEV" in tech:
        return "Freight LT BEV"
    elif "MDV_T_FC" in tech:
        return "MT FCEV"
    elif "HDV_T_FC" in tech:
        return "HT FCEV"
    else: return None

def freeze_after_drop(df):
        df = df.sort_values('period').copy()
        rates = df['rate'].to_numpy()
        # look for first decline
        for i in range(1, len(rates)):
            if rates[i] < rates[i-1]:
                peak = rates[i-1]
                rates[i:] = peak
                break
        df['rate'] = rates
        return df

def table_writer(df, sheet, constraints_dir='trn_constraints_ref2.xlsx'):
    table = pd.read_excel(constraints_dir, sheet_name=sheet)
    df = df[table.columns.tolist()]                                                                            # order columns in the correct format
    with pd.ExcelWriter(constraints_dir, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet, index=False)

def export_deployment(constraints_dir='trn_constraints_ref2.xlsx', scenario='Median', params=growth_params, 
                      p0=2021, pf=2050, model='logistic', capacity_type='net', truncate_growth=None,
                      market_pull_restrict=True, precision=9):
    
    data = deployment_curves(params, p0, pf, model, capacity_type, market_pull_restrict)
    data = data[data['scenario'] == scenario].rename(columns={'rate': 'k_rate'}).copy()
    data['group_name'] = data['tech'].apply(map_tech_to_group)

    seed_df = data.rename(columns={'c0_' + capacity_type: 'seed'})
    seed_df.loc[:, 'seed'] = seed_df.groupby('class')['seed'].transform('min')
    seed = seed_df[seed_df['group_name'] == 'Single'][['tech', 'seed']].drop_duplicates().reset_index(drop=True)
    seed = seed.assign(region='ON', units='k units', notes=f'1% of {capacity_type} capacity from first period, reseeded every period to avoid zeroing out undeployed techs in earlier periods')
    table_writer(seed, 'GrowthRateSeed', constraints_dir)

    gseed = seed_df[seed_df['group_name'] != 'Single'][['group_name', 'seed']].drop_duplicates().reset_index(drop=True)
    gseed = gseed.assign(region='ON', units='k units', notes=f'1% of {capacity_type} capacity from first period, reseeded every period to avoid zeroing out undeployed techs in earlier periods')
    table_writer(gseed, 'GroupGrowthRateSeed', constraints_dir)

    growth = data[(data['group_name'] == 'Single') & (data['period'].isin([2021, 2025, 2030, 2035, 2040, 2045, 2050]))].reset_index(drop=True)
    growth['rate'] = growth.groupby(['tech'], as_index=False)['capacity'].pct_change().round(precision)
    if truncate_growth is True:                                 # freezes growth rates after the first drop
        growth = growth.groupby('tech', group_keys=False) \
                       .apply(freeze_after_drop)
    elif isinstance(truncate_growth, (int, float)):             # manually sets lower bound for growth rates
        growth['rate'] = growth['rate'].clip(lower=truncate_growth)
    growth['notes'] = growth.apply(lambda row: f"Derived from projected {capacity_type} capacity simulated with a {model} diffusion model; "
                                   f"{row['scenario']} growth scenario (where initial_cap={row['c0_' + capacity_type]}, emergence_rate={row['k_rate']}, saturation={row['L_' + capacity_type]})", axis=1)
    growth_out = growth.copy()
    growth = growth[['tech', 'period', 'rate', 'notes']].assign(region='ON').copy()
    growth = growth[growth['period'] != 2021].copy()
    table_writer(growth, 'GrowthRateMax', constraints_dir)

    g_growth = data[(data['group_name'] != 'Single') & (data['period'].isin([2021, 2025, 2030, 2035, 2040, 2045, 2050]))].reset_index(drop=True)
    g_growth['rate'] = g_growth.groupby(['group_name'], as_index=False)['capacity'].pct_change().round(precision)
    if truncate_growth is True:                                 # freezes growth rates after the first drop
        g_growth = g_growth.groupby('tech', group_keys=False) \
                       .apply(freeze_after_drop)
    elif isinstance(truncate_growth, (int, float)):             # manually sets lower bound for growth rates
        g_growth['rate'] = g_growth['rate'].clip(lower=truncate_growth)
    g_growth['notes'] = g_growth.apply(lambda row: f"Derived from projected {capacity_type} capacity simulated with a {model} diffusion model; " 
                                       f"{row['scenario']} growth scenario (where initial_cap={row['c0_' + capacity_type]}, emergence_rate={row['k_rate']}, saturation={row['L_' + capacity_type]})", axis=1)
    g_growth = g_growth[['group_name', 'period', 'rate', 'notes']].assign(region='ON').copy()
    g_growth = g_growth[g_growth['period'] != 2021].copy()
    table_writer(g_growth, 'GroupGrowthRateMax', constraints_dir)  

    return growth_out

In [45]:
def export_deployment_group(constraints_dir='trn_constraints_ref2.xlsx', scenario='Median', params=growth_params, 
                                 p0=2021, pf=2050, model='logistic', capacity_type='net', truncate_growth=None,
                                 market_pull_restrict=True, precision=9):
    
    data = deployment_curves(params, p0, pf, model, capacity_type, market_pull_restrict)
    data = data[data['scenario'] == scenario].rename(columns={'rate': 'k_rate'}).copy()
    data['group_name'] = data['class'] + ' ' + data['powertrain']

    seed_df = data.rename(columns={'c0_' + capacity_type: 'seed'})
    seed_df.loc[:, 'seed'] = seed_df.groupby('class')['seed'].transform('min')
    gseed = seed_df[['group_name', 'seed']].drop_duplicates().reset_index(drop=True)
    gseed = gseed.assign(region='ON', units='k units', notes=f'1% of {capacity_type} capacity from first period, reseeded every period to avoid zeroing out undeployed techs in earlier periods')
    table_writer(gseed, 'GroupGrowthRateSeed', constraints_dir)

    g_growth = data[data['period'].isin([2021, 2025, 2030, 2035, 2040, 2045, 2050])].reset_index(drop=True)
    g_growth['rate'] = g_growth.groupby(['group_name'], as_index=False)['capacity'].pct_change().round(precision)
    if truncate_growth is True:                                 # freezes growth rates after the first drop
        g_growth = g_growth.groupby('tech', group_keys=False) \
                       .apply(freeze_after_drop)
    elif isinstance(truncate_growth, (int, float)):             # manually sets lower bound for growth rates
        g_growth['rate'] = g_growth['rate'].clip(lower=truncate_growth)
    g_growth['notes'] = g_growth.apply(lambda row: f"Derived from projected {capacity_type} capacity simulated with a {model} diffusion model, 5-year growth is frozen before first drop in projections; " 
                                       f"{row['scenario']} growth scenario (where initial_cap={row['c0_' + capacity_type]}, emergence_rate={row['k_rate']}, saturation={row['L_' + capacity_type]})", axis=1)
    g_growth_out = g_growth.copy()
    g_growth = g_growth[['group_name', 'period', 'rate', 'notes']].assign(region='ON').copy()
    g_growth = g_growth[g_growth['period'] != 2021].copy()
    table_writer(g_growth, 'GroupGrowthRateMax', constraints_dir)  

    return g_growth_out

In [ ]:
export_deployment_group(
    constraints_dir='trn_constraints_lowgrowth.xlsx',
    scenario='25th percentile',
    capacity_type='net',
    truncate_growth=True
)

C:\Users\rashi\AppData\Local\Temp\ipykernel_35704\778199836.py:19: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,tech,class,powertrain,max_pull,c0_net,L_net,scenario,k_rate,period,capacity,group_name,rate,notes
0,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2021,96.504725,Cars Hybrid,NaN,Derived from projected net capacity simulated ...
1,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2025,189.657015,Cars Hybrid,0.965261,Derived from projected net capacity simulated ...
2,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2030,434.596907,Cars Hybrid,1.291489,Derived from projected net capacity simulated ...
3,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2035,960.387941,Cars Hybrid,1.291489,Derived from projected net capacity simulated ...
4,T_LDV_C_GSL_HEV_N,Cars,Hybrid,3070.0,96.504725,7514.810859,25th percentile,0.187163,2040,1968.281440,Cars Hybrid,1.291489,Derived from projected net capacity simulated ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,25th percentile,0.187163,2030,19.231519,Pass Light-T Fuel-cell,1.355334,Derived from projected net capacity simulated ...
164,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,25th percentile,0.187163,2035,45.226103,Pass Light-T Fuel-cell,1.355334,Derived from projected net capacity simulated ...
165,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,25th percentile,0.187163,2040,105.968552,Pass Light-T Fuel-cell,1.355334,Derived from projected net capacity simulated ...
166,T_LDV_LTP_FCEV400_N,Pass Light-T,Fuel-cell,2394.0,4.112463,8112.435884,25th percentile,0.187163,2045,246.188477,Pass Light-T Fuel-cell,1.355334,Derived from projected net capacity simulated ...


: 

In [ ]:
def export_deployment_twogroups(constraints_dir='trn_constraints_ref2.xlsx', scenario='Median', params=growth_params, 
                                 p0=2021, pf=2050, model='logistic', capacity_type='net', truncate_growth=None,
                                 market_pull_restrict=True, precision=9):
    
    data = deployment_curves(params, p0, pf, model, capacity_type, market_pull_restrict)
    data = data[data['scenario'] == scenario].rename(columns={'rate': 'k_rate'}).copy()
    data['group_name'] = data['class'] + ' ' + data['powertrain']

    seed_df = data.rename(columns={'c0_' + capacity_type: 'seed'})
    seed_df.loc[:, 'seed'] = seed_df.groupby('class')['seed'].transform('min')
    gseed = seed_df[['group_name', 'seed']].drop_duplicates().reset_index(drop=True)
    gseed = gseed.assign(region='ON', units='k units', notes=f'1% of {capacity_type} capacity from first period, reseeded every period to avoid zeroing out undeployed techs in earlier periods')
    table_writer(gseed, 'GroupGrowthRateSeed', constraints_dir)

    g_growth = data[data['period'].isin([2021, 2025, 2030, 2035, 2040, 2045, 2050])].reset_index(drop=True)
    g_growth['rate'] = g_growth.groupby(['group_name'], as_index=False)['capacity'].pct_change().round(precision)
    if truncate_growth is True:                                 # freezes growth rates after the first drop
        g_growth = g_growth.groupby('tech', group_keys=False) \
                       .apply(freeze_after_drop)
    elif isinstance(truncate_growth, (int, float)):             # manually sets lower bound for growth rates
        g_growth['rate'] = g_growth['rate'].clip(lower=truncate_growth)
    g_growth['notes'] = g_growth.apply(lambda row: f"Derived from projected {capacity_type} capacity simulated with a {model} diffusion model; " 
                                       f"{row['scenario']} growth scenario (where initial_cap={row['c0_' + capacity_type]}, emergence_rate={row['k_rate']}, saturation={row['L_' + capacity_type]})", axis=1)
    g_growth_out = g_growth.copy()
    g_growth = g_growth[['group_name', 'period', 'rate', 'notes']].assign(region='ON').copy()
    g_growth['sub_group_name'] = g_growth['group_name'] + '_new'
    g_growth = g_growth[g_growth['period'] != 2021].copy()
    table_writer(g_growth, 'TwoGroupGrowthRateMax', constraints_dir)  

    return g_growth_out

In [34]:
# export_deployment(
#     constraints_dir='trn_constraints_ref2_newgrowth.xlsx',
#     scenario='Median',
#     capacity_type='new',
#     truncate_growth=1.0
# )

In [35]:
# export_deployment_twogroups(
#     constraints_dir='trn_constraints_ref2_new(net)growth.xlsx',
#     scenario='Median',
#     capacity_type='net',
#     truncate_growth=1.0
# )